In [1]:
import pandas as pd

def load_conll2003(file_path):
    # Define column names based on your CoNLL file structure
    col_names = ['Word', 'POS', 'Chunk', 'Tag']

    data = pd.read_csv(file_path, sep=' ', names=col_names, skip_blank_lines=False)
    data['Sentence_ID'] = (data['Word'].isna()).cumsum()
    return data


file_path = 'project-3-at-2025-07-27-18-26-d57d8db3.conll'
df = load_conll2003(file_path)
print(df.head(20))

def convert_to_text(df):
    df_filtered = df[df['Word'] != '-DOCSTART-']

    lines = df_filtered.apply(lambda row: f"{row['Word']} {row['Tag']}", axis=1)

    with open('output.txt', 'w') as f:
        for line in lines:
            f.write(line + '\n')
convert_to_text(df)

           Word  POS Chunk     Tag  Sentence_ID
0    -DOCSTART-  -X-     O     NaN            0
1        During  -X-     _       O            0
2           the  -X-     _       O            0
3          2015  -X-     _       O            0
4       Ukraine  -X-     _  B-Tool            0
5      Electric  -X-     _  I-Tool            0
6         Power  -X-     _  I-Tool            0
7        Attack  -X-     _       O            0
8             ,  -X-     _       O            0
9      Sandworm  -X-     _       O            0
10         Team  -X-     _       O            0
11     modified  -X-     _       O            0
12  in-registry  -X-     _       O            0
13     internet  -X-     _       O            0
14     settings  -X-     _       O            0
15           to  -X-     _       O            0
16        lower  -X-     _       O            0
17     internet  -X-     _       O            0
18     security  -X-     _       O            0
19            .  -X-     _       O      

In [1]:
#安裝必要套件~~耐心等待喔
!pip3 install datasets seqeval evaluate huggingface transformers accelerate

  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 38.5 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.4
    Uninstalling huggingface-hub-0.33.4:
      Successfully uninstalled huggingface-hub-0.33.4
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3


In [2]:
!git clone https://github.com/stwater20/AIS3-2025-Material.git
%cd AIS3-2025-Material

Cloning into 'AIS3-2025-Material'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 26 (delta 6), reused 25 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 1.57 MiB | 4.11 MiB/s, done.
Resolving deltas: 100% (6/6), done.
/content/AIS3-2025-Material


In [3]:
from datasets import Dataset, DatasetDict, ClassLabel, Sequence, Features
import pandas as pd

In [4]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
!pwd

/content/AIS3-2025-Material


In [6]:
# 整理資料集
def load_data(file_path):
  tokens = []
  ner_tags = []
  temp_tokens = []
  temp_ner_tags = []
  with open(file_path, 'r', encoding='utf-8') as file:
      for line in file:
          if len(line.split())>1:
              line = line.strip()
              token, tag = line.rsplit(' ', 1)
              temp_tokens.append(token)
              temp_ner_tags.append(tag)
          else:
              tokens.append(temp_tokens)
              ner_tags.append(temp_ner_tags)
              temp_tokens = []
              temp_ner_tags = []
  ds = Dataset.from_dict({'tokens': tokens, 'ner_tags': ner_tags})
  return ds

ds_train = load_data('CTI_Basic/Datasets/DNRTI/train.txt')
ds_valid = load_data('CTI_Basic/Datasets/DNRTI/valid.txt')
ds_test = load_data('CTI_Basic/Datasets/DNRTI/test.txt')

In [7]:
# 有時候我們不知道 Tag 有哪幾類, Set 的目的會取出唯一的值
unique_tags = set()
for dataset in [ds_train, ds_valid, ds_test]:
    unique_tags.update([tag for sublist in dataset['ner_tags'] for tag in sublist])
unique_tags = sorted(list(unique_tags))
# unique_tags

In [8]:
# 模型訓練時是告訴模型對應的數字是多少，這可以減少記憶體使用量, 當然之後為了要給人看需要還原回去
label2id = {label: idx for idx, label in enumerate(unique_tags)}
id2label = {idx: label for label, idx in label2id.items()}

In [9]:
def update_ner_tags(example):
    # 更新NER標籤：將標籤轉換為相應的ID
    return {'ner_tags': [label2id[tag] for tag in example['ner_tags']]}

# 使用update_ner_tags函數映射數據集中的每個樣本，將標籤轉為ID
# map是將傳入的函數應用於可迭代對象的每個元素上，並返回一個新的迭代器，其中包含了應用函數後的結果。
ds_train = ds_train.map(update_ner_tags)
ds_valid = ds_valid.map(update_ner_tags)
ds_test = ds_test.map(update_ner_tags)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [10]:
# DatasetDict 可以方便之後的操作 (應用變換)
# https://huggingface.co/docs/datasets/v2.20.0/en/package_reference/main_classes#datasets.DatasetDict
raw_datasets = DatasetDict({
    'train': ds_train,
    'valid': ds_valid,
    'test': ds_test
})
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 5439
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 668
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 679
    })
})

In [11]:
# 印出資料集的 Sample
words = raw_datasets["train"][0]["tokens"]
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = id2label[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

The admin@338 has largely targeted organizations involved in financial , economic and trade  policy , typically using publicly available RATs   such as Poison Ivy    , as well some non-public backdoors . 
O   B-HackOrg O   O       O        O             O        O  B-Idus    O B-Idus   O   B-Idus I-Idus O O         O     B-Tool   I-Tool    I-Tool O    O  B-Tool I-Tool O O  O    O    B-Tool     I-Tool    O 


In [12]:
# 下載分詞器準備處理文字

from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [13]:
#分詞出的結果, 會以 ## 表示原本與前面的單詞合併是原始單詞
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()[:5]

['[CLS]', 'The', 'ad', '##min', '@']

In [14]:
# 將標籤（labels）與分詞後的tokens對齊
# 原始數據集中的標籤通常是針對整個詞的，但分詞器可能將詞拆分成多個子詞（subwords），所以需要重新對標籤進行對齊以保持一致性。
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            new_labels.append(-100)
        else:
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [15]:
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[26, 3, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 26, 26, 11, 24, 26, 26, 26, 26, 11, 24, 26]
[-100, 26, 3, 4, 4, 4, 4, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 24, 24, 26, 26, 11, 12, 24, 26, 26, 26, 26, 11, 12, 12, 24, 24, 24, 26, -100]


In [16]:
#為了預處理整個資料集，我們需要對所有輸入進行標記並應用於align_labels_with_tokens()所有標籤。
#利用快速分詞器同時對大量文字進行分詞，所以我寫了個處理範例清單的函數，並使用Dataset.map()帶有選項的方法batched=True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i) # 對於分詞結果中的每個樣本，利用tokenized_inputs.word_ids(i)獲取每個token對應的原始詞的索引。
        new_labels.append(align_labels_with_tokens(labels, word_ids)) # 將每個詞的NER標記對應到分詞後產生的tokens上, 加回新的 labels list

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [17]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True, #批量處理，一次處理多個樣本，提高效率
    remove_columns=raw_datasets["train"].column_names, # 移除原始數據集中的所有列，只保留處理後的數據
)

Map:   0%|          | 0/5439 [00:00<?, ? examples/s]

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Map:   0%|          | 0/679 [00:00<?, ? examples/s]

In [18]:
from transformers import DataCollatorForTokenClassification

# 創建一個數據整理器的實例，並將分詞器作為參數傳遞，能夠自動對短於最大序列長度的樣本進行填充，並將超過最大長度的樣本進行截斷
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [19]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(2)])
batch["labels"]

tensor([[-100,   26,    3,    4,    4,    4,    4,   26,   26,   26,   26,   26,
           26,    4,   26,    4,   26,    4,   17,   26,   26,   26,   11,   24,
           24,   24,   24,   26,   26,   11,   12,   24,   26,   26,   26,   26,
           11,   12,   12,   24,   24,   24,   26, -100],
        [-100,   26,    3,    4,    4,    4,    4,   26,   26,    0,   13,    6,
           19,   26,   26,   26,   26,   26,    4,   26,    4,   26,   26,    0,
           13,   26,    0,   26, -100, -100, -100, -100, -100, -100, -100, -100,
         -100, -100, -100, -100, -100, -100, -100, -100]])

In [20]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 26, 3, 4, 4, 4, 4, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, 4, 17, 26, 26, 26, 11, 24, 24, 24, 24, 26, 26, 11, 12, 24, 26, 26, 26, 26, 11, 12, 12, 24, 24, 24, 26, -100]
[-100, 26, 3, 4, 4, 4, 4, 26, 26, 0, 13, 6, 19, 26, 26, 26, 26, 26, 4, 26, 4, 26, 26, 0, 13, 26, 0, 26, -100]


In [21]:
import evaluate
# seqeval 是一個常用於評估序列標記任務的指標，如命名實體識別（NER）、詞性標注（POS tagging）等。它可以計算多個常用的評估指標，包括精確度（Precision）、召回率（Recall）和 F1 分數。
metric = evaluate.load("seqeval")

In [22]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [id2label[i] for i in labels]
labels

['O',
 'B-HackOrg',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Idus',
 'O',
 'B-Idus',
 'O',
 'B-Idus',
 'I-Idus',
 'O',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'I-Tool',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'O',
 'O',
 'O',
 'O',
 'B-Tool',
 'I-Tool',
 'O']

In [23]:
predictions = labels.copy()
predictions[1] = "O"
metric.compute(predictions=[predictions], references=[labels])

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'HackOrg': {'precision': np.float64(0.0),
  'recall': np.float64(0.0),
  'f1': np.float64(0.0),
  'number': np.int64(1)},
 'Idus': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(3)},
 'Tool': {'precision': np.float64(1.0),
  'recall': np.float64(1.0),
  'f1': np.float64(1.0),
  'number': np.int64(3)},
 'overall_precision': np.float64(1.0),
 'overall_recall': np.float64(0.8571428571428571),
 'overall_f1': np.float64(0.923076923076923),
 'overall_accuracy': 0.967741935483871}

In [24]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1) # 使用 argmax 獲得最有可能的標籤索引

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[id2label[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [id2label[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [25]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
model.config.num_labels

27

In [28]:
from transformers import TrainingArguments
# 初始化訓練參數
args = TrainingArguments(
    "bert-finetuned-ner",          # 訓練過程中保存模型和結果的目錄名稱
    eval_strategy="epoch",         # 這個跑不了就跑下面那行
    # evaluation_strategy="epoch",   # 設定評估策略為每一個epoch後進行一次評估
    save_strategy="epoch",         # 設定保存策略為每一個epoch後保存模型
    learning_rate=2e-5,            # 設定學習率
    num_train_epochs=3,            # 設定訓練的總epoch數
    weight_decay=0.01,             # 設定權重衰減，用於避免過擬合
    report_to="none",
    push_to_hub=True,              # 設定是否將訓練好的模型推送到 Hugging Face 的 Model Hub
)

In [29]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipython-input-29-3146646831.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.764900,0.454414,0.687329,0.728263,0.707204,0.859359
2,0.411500,0.399864,0.729799,0.777608,0.752945,0.880688
3,0.254300,0.393527,0.745650,0.802110,0.772850,0.889177


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=2040, training_loss=0.43242603049558753, metrics={'train_runtime': 330.2589, 'train_samples_per_second': 49.407, 'train_steps_per_second': 6.177, 'total_flos': 489718522873548.0, 'train_loss': 0.43242603049558753, 'epoch': 3.0})

In [30]:
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/ShengShan/bert-finetuned-ner/commit/4c379b5cad79e2c4458e135581c7b2d612b62a06', commit_message='Training complete', commit_description='', oid='4c379b5cad79e2c4458e135581c7b2d612b62a06', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ShengShan/bert-finetuned-ner', endpoint='https://huggingface.co', repo_type='model', repo_id='ShengShan/bert-finetuned-ner'), pr_revision=None, pr_num=None)

In [31]:
trainer.evaluate(tokenized_datasets["test"])

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'eval_loss': 0.29401376843452454,
 'eval_precision': 0.7948842480168367,
 'eval_recall': 0.8417623864220812,
 'eval_f1': 0.8176519567027477,
 'eval_accuracy': 0.9147920927062031,
 'eval_runtime': 2.7125,
 'eval_samples_per_second': 250.324,
 'eval_steps_per_second': 31.337,
 'epoch': 3.0}

In [32]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "ShengShan/bert-finetuned-ner"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier(["Eset‍ has published a report on the state-sponsored Russian turla apt group."])

config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/669k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Device set to use cuda:0


[[{'entity_group': 'SecTeam',
   'score': np.float32(0.9914315),
   'word': 'E',
   'start': 0,
   'end': 1},
  {'entity_group': 'Time',
   'score': np.float32(0.9870552),
   'word': '##set',
   'start': 1,
   'end': 4},
  {'entity_group': 'Area',
   'score': np.float32(0.8781047),
   'word': 'Russian',
   'start': 52,
   'end': 59},
  {'entity_group': 'HackOrg',
   'score': np.float32(0.5534534),
   'word': 't',
   'start': 60,
   'end': 61},
  {'entity_group': 'Idus',
   'score': np.float32(0.4363737),
   'word': '##la',
   'start': 63,
   'end': 65},
  {'entity_group': 'HackOrg',
   'score': np.float32(0.6012908),
   'word': '##pt group',
   'start': 67,
   'end': 75}]]